In [ ]:
!pip install deepxde

In [ ]:
import deepxde as dde
dde.backend.set_default_backend("pytorch")
from deepxde.backend import torch, backend_name
import torch

import numpy as np
import matplotlib.pyplot as plt

'''
"# DeepXDE will internally create tf.keras layers with these specs."
If there is any issue with torch.sin or other torch.###,
just replace it with tf.sin or tf.###.
'''

# # For reproducibility: fix all random seeds used by DeepXDE / TensorFlow backend
# dde.config.set_random_seed(1234)

In [ ]:
def reference_u(x):
    """
    Reference (true) solution u(x) used to generate synthetic data.

    Here we choose:
        u(x) = sin(pi * x)

    This is the function that the PINN will try to recover from noisy / sampled data,
    and it also implicitly defines the source term q(x) through the Poisson equation.
    """
    return np.sin(np.pi * x)


def reference_q(x):
    """
    Reference source term q(x) corresponding to the chosen u(x).

    Assume the 1D Poisson equation:
        -u''(x) + q(x) = 0

    With u(x) = sin(pi * x):
        u''(x) = -pi^2 * sin(pi * x)

    So plugging into -u''(x) + q(x) = 0 → q(x) = -u''(x) = pi^2 * sin(pi * x).

    NOTE: Here we define:
        reference_q(x) = -pi^2 * sin(pi * x)
    which matches the sign convention used later in the PDE definition
    (-u_xx + q = 0) → q = u_xx.
    """
    return -np.pi**2 * np.sin(np.pi * x)


# Define the 1D spatial domain: x ∈ [-1, 1]
domain = dde.geometry.Interval(-1, 1)


def generate_observation_data(n_points=100):
    """
    Generate synthetic observation data (x_obs, u_obs) from the reference solution.

    Args:
        n_points : number of sample points in the domain.

    Returns:
        x_values : shape (n_points, 1), evenly spaced points in [-1, 1].
        u_values : shape (n_points, 1), reference_u evaluated at x_values.
    """
    # Data generation
    x_values = np.linspace(-1, 1, n_points).reshape(-1, 1)
    u_values = reference_u(x_values)
    return x_values, u_values

# Synthetic "measured" data for the inverse problem
x_observed, u_observed = generate_observation_data()


def is_on_boundary(x, on_boundary):
    """
    Boundary indicator function for DeepXDE.

    Args:
        x           : location in the domain (here, 1D coordinate).
        on_boundary : True if the point is on the boundary of the domain.

    Here we simply return `on_boundary`, meaning we enforce this BC on
    ALL boundary points (both x = -1 and x = 1).
    """
    return on_boundary


def boundary_u(x):
    """
    Boundary value function for Dirichlet BC.

    We impose:
        u(x) = reference_u(x)  on the boundary

    Since reference_u(x) = sin(pi * x), this enforces the exact solution
    at x = -1 and x = 1.
    """
    return reference_u(x)


# Dirichlet boundary condition: u(x) = boundary_u(x) on ∂Ω
boundary_condition = dde.icbc.DirichletBC(
    domain,
    boundary_u,      # target value on the boundary
    is_on_boundary,  # function that selects boundary points
    component=0      # apply this BC to y[:, 0] (the u component)
)


# PointSetBC ~ Class instance: 'x_observed' and 'u_observed' are saved in 'PointSetBC'
# Point-wise observational data as an additional "BC" (data constraint)
"""
x_observed : shape (N, 1)
   - N observation points in the domain
   - each row is a spatial coordinate x where u(x) was measured

 u_observed : shape (N, 1)
   - measured values of u at those locations
   - u_observed[i] corresponds to u(x_observed[i])

class PointSetBC(BoundaryCondition):
    def __init__(self, X, Y, component=0, **kwargs):
        self.X = np.array(X)  # observation (input)
        self.Y = np.array(Y)  # observed value (a specific output from Network)
        self.component = component

    => 'component=0' means that model should be trained
        in a way that y_pred[:,0] be close to u_observed based on MSE loss
"""
observed_data = dde.icbc.PointSetBC(
    x_observed,  # observation locations in the domain (input points)
    u_observed,  # observed u(x) values at those locations (targets)
    component=0  # tell DeepXDE this constraint applies to the 0th output (u, not q)
)


def poisson_inverse_pde(x, y):
    """
    PDE residual for the inverse Poisson problem.

    *** Unknowns: u and q
        y[:, 0:1] = u(x)  (state / solution)
        y[:, 1:2] = q(x)  (unknown source term we want to infer)

    We assume the governing equation:
        -u''(x) + q(x) = 0

    DeepXDE expects a residual F(x, y, ...) such that F = 0 in the domain.

    Steps:
        - extract u and q from y
        - compute u_xx = ∂²u/∂x² via dde.grad.hessian
        - return residual R = -u_xx + q
    """

    # u(x) is the first output of the network
    u = y[:, 0:1]

    # q(x) is the second output of the network
    q = y[:, 1:2]

    # Second derivative of u w.r.t. x: u_xx
    # component=0 → use y[:, 0] (u); i=0, j=0 → derivative w.r.t. x (since x is 1D here)
    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)

    # Residual: -u_xx + q = 0
    return -u_xx + q


# Define the PDE inverse problem: domain + PDE + BCs + training/data settings
pde_problem = dde.data.PDE(
    domain,                     # geometry (1D interval)
    poisson_inverse_pde,        # PDE residual function
    [boundary_condition,        # Dirichlet BC on u
     observed_data],            # point-wise observation constraints on u
    num_domain=200,             # interior collocation points for PDE
    num_boundary=2,             # number of random boundary points
    anchors=x_observed,         # force collocation points to include observed points
    num_test=1000,              # number of test points for evaluation
)


# Neural network architecture: Physics-Informed Feedforward NN (PFNN)
# This network outputs [u(x), q(x)] simultaneously.
neural_net = dde.nn.PFNN(
    [1,                # input dimension: x
     [20, 20],         # hidden block 1
     [20, 20],         # hidden block 2
     [20, 20],         # hidden block 3
     2],               # output dimension: 2 (u, q)
    "tanh",            # activation function
    "Glorot uniform"   # weight initializer
)

# Build DeepXDE model from the PDE data + neural network
model = dde.Model(pde_problem, neural_net)

# Compile model with Adam optimizer
# loss_weights:
#   [1, 100, 1000] correspond to:
#     - PDE residual loss (weight 1)
#     - boundary condition loss (weight 100)
#     - observation data loss (weight 1000)
#   → we emphasize fitting the observed data strongest, then BCs, then PDE.
model.compile("adam", lr=1e-4, loss_weights=[1, 100, 1000])

# Train the model for 20,000 iterations
model.train(iterations=20000)



In [ ]:
x_test = domain.uniform_points(500)
y_pred = model.predict(x_test)

u_predicted = y_pred[:, 0:1]
q_predicted = y_pred[:, 1:2]

u_actual = reference_u(x_test)
q_actual = reference_q(x_test)

error_u = dde.metrics.l2_relative_error(u_actual, u_predicted)
error_q = dde.metrics.l2_relative_error(q_actual, q_predicted)
print(f"L2 Relative Error in u(x): {error_u:.4e}")
print(f"L2 Relative Error in q(x): {error_q:.4e}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(x_test, u_actual, label="u_true", linewidth=2)
plt.plot(x_test, u_predicted, "--", label="u_predicted (NN)")
plt.title("Recovered Displacement u(x)")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(x_test, q_actual, label="q_true", linewidth=2)
plt.plot(x_test, q_predicted, "--", label="q_predicted (NN)")
plt.title("Recovered Forcing Field q(x)")
plt.legend()
plt.tight_layout()
plt.show()


plt.figure()
plt.plot(x_test, np.abs(u_actual - u_predicted), label="|u_true - u_predicted|")
plt.plot(x_test, np.abs(q_actual - q_predicted), label="|q_true - q_predicted|")
plt.title("Point-wise Absolute Errors")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle


def draw_neural_net_with_labels(ax, layer_sizes,
                                node_labels=None,
                                neuron_radius=0.03,
                                connection_color='k',
                                connection_width=0.5):
  """
  Draw a feedforward neural network with customizable neuron labels and layer sizes.
  Parameters:
  ax : Matplotlib Axes object to draw on
  layer_sizes : List of integers indicating neurons in each layer
  node_labels : 2D list of LaTeX-compatible labels (one list per layer)
  neuron_radius : Radius of neuron circles
  connection_color : Color of lines connecting layers
  connection_width : Line width for connections
  """

  v_spacing = 1. / float(max(layer_sizes))
  h_spacing = 1. / float(len(layer_sizes) - 1)

  # Draw neurons
  for layer_idx, num_neurons in enumerate(layer_sizes):
  layer_top = v_spacing * (num_neurons - 1) / 2.0
  for neuron_idx in range(num_neurons):
  x = layer_idx * h_spacing
  y = layer_top - neuron_idx * v_spacing
  circle = Circle((x, y), neuron_radius, edgecolor='k', facecolor='w', zorder=4)
  ax.add_patch(circle)

  # Add labels if provided
  if node_labels and layer_idx < len(node_labels) and neuron_idx < len(node_labels[
  ax.text(x, y, node_labels[layer_idx][neuron_idx],
  ha='center', va='center', fontsize=10, color='black', zorder=5)

  # Draw connections between neurons
  for layer_idx, (n_input, n_output) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
  layer_top_input = v_spacing * (n_input - 1) / 2.0
  layer_top_output = v_spacing * (n_output - 1) / 2.0
  for i in range(n_input):
  for j in range(n_output):
  x0 = layer_idx * h_spacing
  y0 = layer_top_input - i * v_spacing
  x1 = (layer_idx + 1) * h_spacing
  y1 = layer_top_output - j * v_spacing
  ax.plot([x0, x1], [y0, y1], color=connection_color, lw=connection_width, zorde



# Example: 1 input → 2 hidden → 2 output (u, q)
layer_sizes = [1, 4, 4, 2]

# Optional: labels per neuron
node_labels = [
[r"$x$"], # Input layer
[r"$h_{11}$", r"$h_{12}$", r"$h_{13}$", r"$h_{14}$"], # Hidden 1
[r"$h_{21}$", r"$h_{22}$", r"$h_{23}$", r"$h_{24}$"], # Hidden 2
[r"$u(x)$", r"$q(x)$"] # Output layer
]

fig, ax = plt.subplots(figsize=(10, 5))
draw_neural_net_with_labels(ax, layer_sizes, node_labels)
ax.axis('off')

plt.title("Neural Network Architecture for Inverse Poisson PINN", fontsize=14)
plt.show()



